### Inception v1

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
# 定义包含conv和ReLU的基本卷积层
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels,
                              out_channels,
                              kernel_size,
                              padding=padding)

    def forward(self, x):
        x = self.conv(x)
        return F.relu(x, inplace=True)


class Inceptionv1(nn.Module):
    def __init__(self, in_dim, hid_1_1, hid_2_1, hid_2_3, hid_3_1, out_3_5,
                 out_4_1):
        super(Inceptionv1, self).__init__()
        self.branch1x1 = BasicConv2d(in_dim, hid_1_1, 1)
        self.branch3x3 = nn.Sequential(
            BasicConv2d(in_dim, hid_2_1, 1),
            BasicConv2d(hid_2_1, hid_2_3, 3, padding=1))
        self.branch5x5 = nn.Sequential(
            BasicConv2d(in_dim, hid_3_1, 1),
            BasicConv2d(hid_3_1, out_3_5, 5, padding=2))
        self.branch_pool = nn.Sequential(nn.MaxPool2d(3, stride=1, padding=1),
                                         BasicConv2d(in_dim, out_4_1, 1))

    def forward(self, x):
        b1 = self.branch1x1(x)
        b2 = self.branch3x3(x)
        b3 = self.branch5x5(x)
        b4 = self.branch_pool(x)
        output = torch.cat((b1, b2, b3, b4), dim=1)
        return output

In [3]:
'''
网络实例化
'''

net_inceptionv1 = Inceptionv1(3, 64, 32, 64, 64, 96, 32)

In [4]:
net_inceptionv1

Inceptionv1(
  (branch1x1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
  )
  (branch3x3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
  (branch5x5): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 64, kernel_size=(1, 1), stride=(1, 1))
    )
    (1): BasicConv2d(
      (conv): Conv2d(64, 96, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    )
  )
  (branch_pool): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
    (1): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1))
    )
  )
)

In [6]:
'''
实例计算
'''

x = torch.randn(1, 3, 256, 256)
y = net_inceptionv1(x)
y.shape

torch.Size([1, 256, 256, 256])

### Inception v2

In [7]:
# 定义卷积基础块，增加了bn层
class BasicConv2d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super(BasicConv2d, self).__init__()
        self.conv = nn.Conv2d(in_channels,
                              out_channels,
                              kernel_size,
                              padding=padding)
        self.bn = nn.BatchNorm2d(out_channels, eps=0.001)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return F.relu(x, inplace=True)


class Inceptionv2(nn.Module):
    def __init__(self):
        super(Inceptionv2, self).__init__()
        self.branch1 = BasicConv2d(192, 96, 1, 0)
        self.branch2 = nn.Sequential(BasicConv2d(192, 48, 1, 0),
                                     BasicConv2d(48, 64, 3, 1))
        self.branch3 = nn.Sequential(BasicConv2d(192, 64, 1, 0),
                                     # 使用两个3X3代替5X5卷积核
                                     BasicConv2d(64, 96, 3, 1),
                                     BasicConv2d(96, 96, 3, 1))
        self.branch4 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1, count_include_pad=False),
            BasicConv2d(192, 64, 1, 0))

    def forward(self, x):
        x0 = self.branch1(x)
        x1 = self.branch2(x)
        x2 = self.branch3(x)
        x3 = self.branch4(x)
        out = torch.cat((x0, x1, x2, x3), 1)
        return out

In [8]:
'''
实例化Inception v2
'''

net_inceptionv2 = Inceptionv2()

In [9]:
net_inceptionv2

Inceptionv2(
  (branch1): BasicConv2d(
    (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
    (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (branch2): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(192, 48, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(48, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (branch3): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicConv2d(
      (conv): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.1, affine=True, track_runnin

In [10]:
'''
实例计算
'''

x = torch.randn(1, 192, 32, 32)
y = net_inceptionv2(x)
y.shape

torch.Size([1, 320, 32, 32])